In [485]:
import pandas as pd
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.initializers import VarianceScaling, Constant
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

import time
import sklearn as skl
import numpy as np


In [140]:
df = pd.read_csv("./Montreal AirBnb/listings.csv")
df2 = pd.read_csv("./Montreal AirBnb/calendar.csv")
df3 = pd.read_csv("./Montreal AirBnb/reviews.csv")

In [260]:
finaldf = pd.DataFrame()

In [415]:
finaldf["id"]= df["id"]
finaldf["lat"] = df["latitude"]
finaldf["long"] = df["longitude"]
finaldf["property_type"]=df["property_type"]
finaldf = labeler("property_type")
finaldf["room_type"] =df["room_type"]
finaldf = labeler("room_type")
finaldf["people"]=df["accommodates"]
finaldf["reviews"] = df["number_of_reviews"]
finaldf["bedrooms"] = df["bedrooms"].fillna(0)
finaldf["bathrooms"]=df["bathrooms"].fillna(0)
finaldf["no. beds"] = df["beds"]
finaldf["response rate"]= df["host_response_rate"].fillna(0)
finaldf["superhost"] = df["host_is_superhost"]
finaldf["verified identity"] = df["host_identity_verified"]
finaldf["neighbourhood"]=df["neighbourhood_cleansed"]
finaldf = labeler("neighbourhood")
finaldf["guests"]= df["guests_included"]
#final["guests charged"] = 
finaldf["min_nights"]=df["minimum_nights"]
finaldf["review rating"] = (df["review_scores_rating"]/100).fillna(0)
finaldf["instant bookable"]=df["instant_bookable"]
finaldf["cancellation_cat"] = df["cancellation_policy"]
finaldf = labeler("cancellation_cat")

#use lat long to count no. of competitiors with same radius(or square window)

#finaldf["price"]=df["price"]

finaldf = finaldf.replace({'superhost': {'t': 1, 'f': 0}}).fillna(0)
finaldf = finaldf.replace({'verified identity': {'t': 1, 'f': 0}}).fillna(0)
finaldf = finaldf.replace({'instant bookable':{'t':1, 'f':0}}).fillna(0)
#finaldf = finaldf.replace({'cancellation policy':{'flexible':1,'moderate':2,'strict':3}})
#finaldf["cancellation_cat"] = finaldf["cancellation_cat"].astype('category')
#finaldf["cancellation_cat"] = finaldf["cancellation_cat"].cat.codes



In [417]:
#df2.fillna(-1)
#final["avg. price"] = 
pandas.set_option('display.max_columns', None)


In [418]:
def labeler(name):
    finaldf[name]=finaldf[name].astype('category')
    finaldf[name]=finaldf[name].cat.codes
    return finaldf

In [419]:
lat = finaldf["lat"]
long = finaldf["long"]


In [420]:
#finaldf.plot(kind = "scatter", x='lat', y='long', figsize=(20,10));

In [421]:
#finaldf.head()

In [422]:
#for i in range(0,111):
#    print("_________")
#    print(df2[(i*365):((i+1)*365)]["price"].dropna().describe())   

In [423]:
pricedf = df2[(df2['date'] > '2016-11-01') & (df2['date'] < '2016-12-01')].dropna()
#pricedf =pricedf["unique" != True]
pricedf["id"]=pricedf["listing_id"]
pricedf["listing_id"]=pricedf["listing_id"].drop_duplicates()
pricedf = pricedf.dropna()
pricedf.head()
result = pd.merge(finaldf, pricedf, on="id")
result = result.drop(['listing_id', 'date', 'available'], axis = 1)
result["price"] = result["price"].replace(["\$", ','],"", regex=True).astype(float)
result["response rate"] = result["response rate"].replace(["%"],"", regex=True).astype(float)/100
result = result.sample(frac=1).reset_index(drop=True)


features = result.drop(["id", "price"],axis =1)

In [466]:
temp = result["price"]
count_nan = len(features) - features.count()
count_nan
result.head()

,id,lat,long,property_type,room_type,people,reviews,bedrooms,bathrooms,no. beds,response rate,superhost,verified identity,neighbourhood,guests,min_nights,review rating,instant bookable,cancellation_cat,price
0,6265611,45.470377,-73.615515,10,1,2,18,1.0,1.0,1.0,1.00,0.0,0,5,1,2,0.87,0,2,30.0
1,9138070,45.520309,-73.561487,0,0,4,0,2.0,1.0,2.0,0.83,0.0,0,29,1,3,0.00,0,0,100.0
2,5236198,45.520119,-73.559368,0,1,2,0,1.0,1.0,1.0,0.00,0.0,0,29,1,1,0.00,0,3,45.0
3,6337948,45.527408,-73.557949,0,1,2,30,1.0,1.0,1.0,1.00,1.0,1,29,1,2,0.98,1,2,55.0
4,7931203,45.559480,-73.571488,0,1,2,3,1.0,1.0,1.0,1.00,0.0,0,24,1,1,1.00,0,0,56.0


# Result
So basically, we have the ID's, lat, long and then a bunch of columns of features(X vector) and then we have Price as the last column which will be our Y

This is done for Nov-Dec 2016.
Now to integrate with the other feature vectors and get the neural net working

In [459]:
#setting x_train, x_test, y_train, y_test 
def setTestTrain(featuresDf, splitLast):
    
    x_train = featuresDf.values[:-splitLast]
    
    y_train = result["price"].values[:-splitLast]
    
    x_test = featuresDf.values[-splitLast:]
    y_test = result["price"].values[-splitLast:]
    
    #x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    #x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  


    return x_train, y_train, x_test, y_test
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 


#building model
def build_model(layer1, layer2, layer3, layer4):
    model = Sequential()
    model.add(Dense(layer1, input_dim=18, activation='selu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(layer2, activation='selu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    

    model.add(Dense(layer3, activation='selu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(layer4, activation='selu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    
    model.add(Dense(1, activation='linear'))
    
    #compile
    start = time.time()
    model.compile(loss=root_mean_squared_error, optimizer="adamax",metrics =["mse"])
    print("> Compilation Time : ", time.time() - start)
    return model

In [491]:
x_train, y_train, x_test, y_test = setTestTrain(features, 1150)
std_x_train = StandardScaler().fit_transform(x_train, y = y_train)
min_x_train = MinMaxScaler(feature_range=(0, 1)).fit_transform(x_train, y = y_train)

model = build_model(512,256,128,64)
model.summary()


> Compilation Time :  0.08079886436462402
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 512)               9728      
_________________________________________________________________
batch_normalization_31 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_47 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_68 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_32 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_48 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_69 (Dense)             (None

In [493]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001, verbose = 2)

callbacks_list = [checkpoint, reduce_lr]

In [ ]:
model.fit(std_x_train, y_train, epochs=100, batch_size=8, validation_split = 0.2, callbacks=callbacks_list, verbose=1)

pca, tsne, check features, go through feature selection algorithms
batch norm, dropout
xgboost, 3 layer 256 , relu, adamax, linear decay small, input then dropout
shuffle before feeding into network


In [486]:
model.predict(x_test)

array([[ 47.66233063],
       [ 41.8419342 ],
       [ 45.76735687],
       ..., 
       [ 41.8520813 ],
       [ 93.5235672 ],
       [ 86.02532959]], dtype=float32)

In [484]:
coords = pd.DataFrame()
coords["lat"] = result["lat"]
coords["long"] = result["long"]
coords.to_csv("coords.csv")

xtrain min max scaling (0,1)

xtrain standard scaling


In [492]:
min_x_train

array([[ 0.199861  ,  0.70238025,  0.73333333, ...,  0.87      ,
         0.        ,  0.66666667],
       [ 0.37612587,  0.82918874,  0.06666667, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.37545306,  0.83416184,  0.06666667, ...,  0.        ,
         0.        ,  1.        ],
       ..., 
       [ 0.44059768,  0.79760421,  0.06666667, ...,  0.96      ,
         0.        ,  1.        ],
       [ 0.2826783 ,  0.82710837,  0.06666667, ...,  0.96      ,
         0.        ,  1.        ],
       [ 0.40604298,  0.80567841,  0.06666667, ...,  0.        ,
         0.        ,  0.        ]])